In [1]:
from fastai.vision.all import *
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model, save_model

c:\Users\Aniket Konkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_seed(7)

In [3]:
train_csv = pd.read_csv('dataset/train.csv')
train_csv = train_csv[train_csv['Image'] != 'w_7489.jpg'] # Removing this entry as this image is not present in the folder
test_csv = pd.read_csv('dataset/sample_submission.csv')
test_image_names = test_csv['Image']
test_image_names.values

array(['w_1947.jpg', 'w_11096.jpg', 'w_10973.jpg', ..., 'w_5278.jpg',
       'w_9218.jpg', 'w_9316.jpg'], dtype=object)

In [4]:
def load_train_images_to_dataframe(folder_path):
    image_data = []
    files = os.listdir(folder_path)
    counter = 0
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)

        if file_name not in test_image_names.values:
            counter += 1
            img = cv2.imread(file_path)
            if img is None:
                print(f"Error: Image {file_name} not loaded!")
                continue
            image_data.append(img)

    df = np.array(image_data)    
    return df

In [6]:
x_train = load_train_images_to_dataframe('dataset/imgs256x256')
y_train = train_csv

In [7]:
print(x_train.shape)
print(y_train.shape)

(4543, 256, 256, 3)
(4543, 2)


In [8]:
# Get unique strings in the column
unique_strings = y_train['whaleID'].unique()

# Create a mapping dictionary to assign values between 0 and 446 to the unique strings
mapping = {string: i for i, string in enumerate(unique_strings)}

# Update the mapping to ensure it assigns the same values to the same strings
updated_mapping = {string: mapping[string] for string in y_train['whaleID'].unique()}

# Assign values to the strings in the column using the updated mapping
y_train['whaleID'] = y_train['whaleID'].map(updated_mapping)

# Display the DataFrame with the assigned values
print(y_train)

           Image  whaleID
0     w_7812.jpg        0
1     w_4598.jpg        1
2     w_3828.jpg        2
3     w_8734.jpg        3
4     w_3251.jpg        4
...          ...      ...
4539  w_9399.jpg      175
4540  w_9403.jpg      311
4541  w_9428.jpg      243
4542  w_9450.jpg      332
4543  w_9468.jpg      287

[4543 rows x 2 columns]


In [9]:
# total number of classes
unique_strings.size 

447

In [10]:
# just need the whaleIDs as target values
y_train = y_train['whaleID']

In [11]:
print(x_train.shape)
print(y_train.shape)

(4543, 256, 256, 3)
(4543,)


In [12]:
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten
model.add(Flatten())

# Add 2 fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dense(447, activation='softmax'))  # Output layer with 447 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\Aniket Konkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    58,982,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 447)            │       229,311 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,305,471 (226.23 MB)

 Trainable params: 59,305,471 (226.23 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 98s 855ms/step - accuracy: 0.0037 - loss: 79.0620 - val_accuracy: 0.0220 - val_loss: 6.0716
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 94s 823ms/step - accuracy: 0.0185 - loss: 6.0078 - val_accuracy: 0.0077 - val_loss: 6.0070
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 94s 827ms/step - accuracy: 0.0426 - loss: 5.8228 - val_accuracy: 0.0044 - val_loss: 6.2961
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 94s 827ms/step - accuracy: 0.3070 - loss: 4.3444 - val_accuracy: 0.0044 - val_loss: 7.4374
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 94s 823ms/step - accuracy: 0.5938 - loss: 2.1779 - val_accuracy: 0.0066 - val_loss: 11.7317
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 95s 832ms/step - accuracy: 0.8062 - loss: 0.9725 - val_accuracy: 0.0066 - val_loss: 14.4193
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 95s 831ms/step - accuracy: 0.8805 - loss: 0.5832 - val_accuracy: 0.0033 - val_loss: 18.3645
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 94s 827ms/step - accuracy: 0.9162 - los

In [14]:
def load_test_images_to_dataframe(folder_path):
    image_data = []
    files = os.listdir(folder_path)
    counter = 0

    # Iterate over each file
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
    
        if file_name in test_image_names.values:
            counter += 1
            img = cv2.imread(file_path)
            if img is None:
                print(f"Error: Image {file_name} not loaded!")
                continue
            
            image_data.append(img)
            
    df = np.array(image_data)
    return df

In [15]:
x_test = load_test_images_to_dataframe('dataset/imgs256x256')
x_test.shape

(6925, 256, 256, 3)

In [16]:
y_pred = model.predict(x_test)

217/217 ━━━━━━━━━━━━━━━━━━━━ 29s 134ms/step


In [17]:
# Find the column index of the maximum value in each row
max_columns = np.argmax(y_pred, axis=1)

# Create a new array filled with zeros
result_arr = np.zeros_like(y_pred)

# Set the value of the highest column to 1 for each row
rows = np.arange(y_pred.shape[0])
result_arr[rows, max_columns] = 1
y_pred_binary = np.where(result_arr > 0, 1, 0)

In [18]:
# Extract the column names from the sorted mapping
column_names = ['Image'] + [key for key, _ in sorted(mapping.items(), key=lambda x: x[1])]

# Create DataFrame with column headers
df = pd.DataFrame(columns=column_names)

# Populate 'Image' column with values from image_column_arr
df['Image'] = test_csv['Image']

# Add binary_arr as values to the DataFrame
df[df.columns[1:]] = y_pred_binary

In [19]:
df.shape

(6925, 448)

In [20]:
df.head()

,Image,whale_48813,whale_09913,whale_45062,whale_74162,whale_99558,whale_59255,whale_87291,whale_33152,whale_88147,...,whale_34513,whale_81915,whale_44747,whale_16762,whale_22848,whale_17327,whale_89271,whale_08729,whale_05140,whale_51114
0,w_1947.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,w_11096.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,w_10973.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,w_10442.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,w_10606.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# write to csv for submission
df.to_csv("submissions/naive-cnn-#2.csv", index=False) 

In [23]:
# save the trained model
# save_model(model, "models/naive-cnn.h5") # Kaggle Public score: 34.23943
save_model(model, "models/naive-cnn-#2.h5") # Kaggle Public score: 34.41408